In [8]:

from ultralytics import YOLO

# Load a model
#model = YOLO(r"C:/Users/Huijb/OneDrive/Bureaublad/Fontys/Semester 4/Group project/YOLOv8/runs/detect/train4/weights/last.pt")  # build a new model from scratch
model = YOLO("yolov8n.yaml")  # build a new model from scratch


# Use the model
model.train(data="config.yaml", epochs=5, batch=6, resume=True, half=True)

#test_image_path = r"C:\Users\Huijb\OneDrive\Bureaublad\Fontys\Semester 4\Group project\YOLOv8\IMG_7738.jpg"  # Replace with the path to a specific image from your dataset

# Run inference on the test image
#results = model.predict(source=test_image_path, save=True)



image 1/1 C:\Users\Huijb\OneDrive\Bureaublad\Fontys\Semester 4\Group project\YOLOv8\IMG_7738.jpg: 384x640 (no detections), 25.7ms
Speed: 4.7ms preprocess, 25.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict
